**DEEP SEEK LLM** **`Finetuning`**

In [1]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

In [2]:
csv_path = "/content/Finance_data.csv"  # Replace with your CSV path
df = pd.read_csv(csv_path)

In [3]:
def row_to_text_auto(row):
    return ", ".join([f"{col}: {row[col]}" for col in row.index])

df['text'] = df.apply(row_to_text_auto, axis=1)

In [4]:
dataset = Dataset.from_pandas(df[['text']])

In [5]:
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"  # Replace with your model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [6]:
# ---- Apply LoRA ----
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # adjust if different in your model
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [7]:
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        max_length=256,  # adjust depending on your text length
        padding="max_length"
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

In [8]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [9]:
# ---- Training ----
training_args = TrainingArguments(
    output_dir="./lora_finetune_investment",
    per_device_train_batch_size=1,
    num_train_epochs=2,
    logging_steps=10,
    fp16=True,
    save_strategy="no"
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

In [11]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kalyansundar (kalyansundar-governmentjobs-com) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,8.549500
20,8.707100
30,8.286700
40,8.186100
50,7.997900
60,7.780900
70,7.779400
80,7.682300


TrainOutput(global_step=80, training_loss=8.121237659454346, metrics={'train_runtime': 63.0925, 'train_samples_per_second': 1.268, 'train_steps_per_second': 1.268, 'total_flos': 157530256834560.0, 'train_loss': 8.121237659454346, 'epoch': 2.0})

In [29]:
def generate_response(prompt, max_length=128):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [42]:
prompt = "give me python code swap 2 numbers"

In [43]:
print("Chatbot response:", generate_response(prompt))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Chatbot response: give me python code swap 2 numbers without using third variable


def swap(a, b):
    a = a + b
    b = a - b
    a = a - b
    return a, b

a = 5
b = 10
a, b = swap(a, b)
print(a, b)

# output: 10 5

# this code works by adding the two numbers together, then subtracting the second number from the result to get the first number, and finally subtracting the new first number from the result to get the second number.
# this is
